# Session 1<br>Introduction to Q# and the Quantum Development Kit
Dr Chris Granade<br>
Senior Research Software Development Engineer<br>
Microsoft

---

Follow along online at [aka.ms/ieee-quantum-2020/session1](https://aka.ms/ieee-quantum-2020/session1).

# Agenda

- What is Q#?
- Functions and operations
- Magic commands
- Types in Q#
- Control flow
- Diagnosing Q# programs
- Putting it all together

# What is Q#?

Let's start with an example.

In [1]:
operation DemonstrateTeleportation() : Result {
    using ((msg, here, there) = (Qubit(), Qubit(), Qubit())) {
        Ry(1.23, msg);
        
        H(here);
        CNOT(here, there);
        
        CNOT(msg, here);
        H(msg);
        
        let xCorrection = M(here);
        let zCorrection = M(msg);
        
        if (xCorrection == One) { X(there); }
        if (zCorrection == One) { Z(there); }
        
        Ry(-1.23, there);
        
        return M(there);
    }
}

DemonstrateTeleportation

In [7]:
%simulate DemonstrateTeleportation

Zero

In [15]:
%trace DemonstrateTeleportation

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

We can make this more modular by extracting the actual teleportation step.

In [16]:
operation TeleportMessage(msg : Qubit, there : Qubit) : Unit {
    using (here = Qubit()) {        
        H(here);
        CNOT(here, there);
        
        CNOT(msg, here);
        H(msg);
        
        if (M(here) == One) { X(there); }
        if (M(msg) == One)  { Z(there); }
    }
}

TeleportMessage

In [17]:
operation DemonstrateTeleportation() : Result {
    using ((msg, there) = (Qubit(), Qubit())) {
        Ry(1.23, msg);        
        TeleportMessage(msg, there);        
        Ry(-1.23, there);
        
        return M(there);
    }
}

DemonstrateTeleportation

In [22]:
%simulate DemonstrateTeleportation

Zero

In [23]:
%trace DemonstrateTeleportation

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

In [38]:
%trace DemonstrateTeleportation --depth=2

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

We can make our teleportation program even more modular.

In [39]:
operation PrepareEntangledPair(left : Qubit, right : Qubit) : Unit is Adj + Ctl {
    H(left);
    CNOT(left, right);
}

PrepareEntangledPair

In [40]:
operation TeleportMessage(msg : Qubit, there : Qubit) : Unit {
    using (here = Qubit()) {        
        PrepareEntangledPair(here, there);
        Adjoint PrepareEntangledPair(msg, here);
        
        ApplyIfOne(M(here), (X, there));
        ApplyIfOne(M(msg),  (Z, there));
    }
}

TeleportMessage

In [41]:
%simulate DemonstrateTeleportation

Zero

In [43]:
%trace DemonstrateTeleportation --depth=2

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

In [44]:
open Microsoft.Quantum.Diagnostics;

operation TeleportPreparedState(preparation : (Qubit => Unit is Adj + Ctl)) : Unit {
    using ((msg, there) = (Qubit(), Qubit())) {
        preparation(msg);        
        TeleportMessage(msg, there);        
        Adjoint preparation(there);
        
        AssertQubit(Zero, there);
    }
}

TeleportPreparedState

In [45]:
operation DemonstrateTeleportation() : Unit {
    TeleportPreparedState(Ry(1.23, _));
}

DemonstrateTeleportation

In [46]:
%simulate DemonstrateTeleportation

()

In [47]:
operation DemonstrateEntanglementSwapping() : Unit {
    using (reference = Qubit()) {
        TeleportPreparedState(PrepareEntangledPair(reference, _));
    }
}

DemonstrateEntanglementSwapping

In [48]:
%simulate DemonstrateEntanglementSwapping

()

# What is Q#?

> Quantum programs are **classical programs** that can send instructions to and get measurement results back from simulators and quantum devices.

From that view, Q# is a classical language designed to make it easier to write, test, and run quantum programs:

- High-level
- Focused on algorithms
- Portable across simulators and hardware
- Quantum-focused features
- Diagnostics to help test and understand

## Use your tools with Q# ##

![](./use-qsharp-with.png)

---

For this demo, we'll use Q# from within Jupyter.

```
$ conda install -c quantum-engineering qsharp
```

Get started using Q# with your tools at [aka.ms/get-started-qdk](https://aka.ms/get-started-qdk).

Let's take a look at the various features of Q#, and how they help us focus back on quantum algorithms.

## Functions and Operations

Q# programs are broken down into two kinds of subroutines:

- **Functions** represent determinstic classical logic <br/>
  e.g.: `Sin`, `Sqrt`

In [49]:
function Squared(x : Double) : Double {
    return x * x;
}

Squared

In [57]:
%simulate Squared x=3.14

9.8596

- **Operations** represent subroutines that can take some action, such as sending instructions to a quantum device <br/>
  e.g.: `H`, `M`, `CNOT`

In [58]:
operation SampleQrng() : Result {
    using (q = Qubit()) {
        return Measure([PauliX], [q]);
    }
}

SampleQrng

In [65]:
%simulate SampleQrng

Zero

## Magic Commands for Q# ##

When working with Q# from a Jupyter Notebook, commands starting with `%` can be used to control the Q# session itself.

In [66]:
%lsmagic

Name,Summary,Assembly
%azure.connect,Connects to an Azure Quantum workspace or displays current connection status.,Microsoft.Quantum.IQSharp.AzureClient
%azure.execute,Executes a job in an Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.jobs,Displays a list of jobs in the current Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.output,Displays results for a job in the current Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.status,Displays status for a job in the current Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.submit,Submits a job to an Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%azure.target,Sets or displays the active execution target for Q# job submission in an Azure Quantum workspace.,Microsoft.Quantum.IQSharp.AzureClient
%config,Allows setting or querying configuration options.,Microsoft.Quantum.IQSharp.Kernel
%estimate,Runs a given function or operation on the ResourcesEstimator target machine.,Microsoft.Quantum.IQSharp.Kernel
%lsmagic,Returns a list of all currently available magic commands.,Microsoft.Quantum.IQSharp.Kernel


We can use `%simulate` to run a function or operation on a full-state quantum simulator.

In [67]:
function Greeting(name : String) : Unit {
    Message($"Hello, {name}!");
}

Greeting

In [68]:
%simulate Greeting name=Chris

Hello, Chris!


()

Other simulators also provide their own magic commands for running functions and operations.

In [69]:
%estimate SampleQrng

Metric,Sum,Max
CNOT,0,0
QubitClifford,0,0
R,0,0
Measure,1,1
T,0,0
Depth,0,0
Width,1,1
BorrowedWidth,0,0


In [70]:
operation DemoMultiControlledX(nQubits : Int) : Unit {
    using (register = Qubit[nQubits]) {
        ApplyToEach(X, register);
        Controlled X(register[...Length(register) - 2], register[Length(register) - 1]);
        DumpMachine();
        ResetAll(register);
    }
}

DemoMultiControlledX

In [71]:
%toffoli DemoMultiControlledX nQubits=1000

Offset  	State Data
========	==========
00000000	FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
00000010	FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
00000020	FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
00000030	FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
00000040	FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
00000050	FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
00000060	FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
00000070	FF FF FF FF FF FF FF FF FF FF FF FF 7F


()

To use additional libraries and simulators, you can use the `%package` command.

In [72]:
%package QSharpCommunity.Simulators.Chp

Adding package QSharpCommunity.Simulators.Chp: done!

Microsoft.Quantum.Standard::0.12.20082513, QSharpCommunity.Simulators.Chp::0.1.7

In [73]:
operation DemoEntangledPreparation() : Unit {
    using ((left, right) = (Qubit(), Qubit())) {
        PrepareEntangledPair(left, right);
        DumpMachine();
        ResetAll([left, right]);
    }
}

DemoEntangledPreparation

In [74]:
%config chp.nQubits = 2
%chp DemoEntangledPreparation

<+XX, +ZZ>| >+ZI, +IX<

()

In [75]:
%simulate DemoEntangledPreparation

|00⟩	0.7071067811865476 + 0𝑖
|11⟩	0.7071067811865476 + 0𝑖

()

## Types in Q#: Classical Data

When we declare functions and operations, we need to say what the **type** of each input and output is.

- **Numeric data types**: `Int`, `BigInt`, `Double`, `Complex`
- **Boolean**: `Bool` (`true` or `false`)
- **Measurement results**: `Result` (`Zero` or `One`)
- **Pauli operators**: `Pauli` (`PauliI`, `PauliX`, `PauliY`, or `PauliZ`)
- **Ranges of integers**: `Range` (e.g.: `0..9`, `10..-1..1`)
- **Diagnostic strings**: `String` (e.g.: `"Hello!"`)

We can declare new _variables_ of each type by using the `let` keyword:

In [76]:
open Microsoft.Quantum.Math;

function AngleForBias(bias : Double) : Double {
    // Declares probability as a Double.
    let probability = 0.5 + bias;
    return 2.0 * ArcCos(Sqrt(1.0 - probability));
}

AngleForBias

## Mutability and Immutability

By default, variables in Q# are **immutable**.

In [77]:
function DoesntWork() : Unit {
    let x = 3; // declare x as an Int
    set x = 10; // error!
}

/snippet_.qs(3,9): error QS6303: An immutable identifier cannot be modified.


To modify variables, use the `mutable` and `set` keywords:

In [78]:
function Works() : Unit {
    mutable x = 3; // declare x as an Int, and allows us to modify it later
    set x = 10;
}

Works

In [79]:
%simulate Works

()

The `set` keyword can also be used with in-place operators, such as `+=`:

In [80]:
function WhereEven(array : Int[]) : Int[] {
    mutable evens = new Int[0];
    for (element in array) {
        if (element % 2 == 0) {
            set evens += [element];
        }
    }
    return evens;
}

function EvensExample() : Unit {
    Message($"{WhereEven([0, 11, 13, 16, 101, 142])}");
}

WhereEven, EvensExample

In [81]:
%simulate EvensExample

[0,16,142]


()

While variables in Q# can be declared as mutable, **values are always immutable**.

You can assign a new value to a mutable variable, but that has no effect on anywhere else that value is used.

In [82]:
function ImmutableValueExample() : Unit {
    mutable a = [3];
    let b = a;
    // Changing a has no effect on the value of b.
    set a = [4, 5];
    
    Message($"a = {a}, b = {b}");
}

ImmutableValueExample

In [83]:
%simulate ImmutableValueExample

a = [4,5], b = [3]


()

## Types in Q#:  Tuples

Given any sequence of types, you can make a new **tuple** of those types using `()`.

In [84]:
operation SampleEntangledPair() : (Result, Result) {
    using ((left, right) = (Qubit(), Qubit())) {
        PrepareEntangledPair(left, right);
        return (M(left), M(right));
    }
}

SampleEntangledPair

In [99]:
%simulate SampleEntangledPair

(Zero, Zero)

### Tuple-in Tuple-out
Every Q# function and operation takes exactly one input, and returns exactly one output.

> 💡 **TIP**: _Singleton–tuple equivalence_
>
> Every tuple with exactly one item is identical to that item on its own.
>
> E.g.: `3` and `(3)` are the same value.

In [100]:
function TupleExample() : Unit {
    let summands = (2, 3);
    Message($"{Fst(summands)} + {Snd(summands)} = {PlusI(summands)}");
    let three = ((3));
    Message($"2 + {three} = {PlusI(2, three)}");
}

TupleExample

In [101]:
%simulate TupleExample

2 + 3 = 5
2 + 3 = 5


()

Like Python and F#, all functions and operations in Q# return a value.

Often, if an operation has no meaningful value to return, it will return the empty tuple `()` of type `Unit`.

> 💡 **TIP**
>
> Unlike `void` in other languages, `Unit` is a bit like an empty box.
>
> There's still a box, it just happens to be empty.

We can make tuples of other types as well:

In [102]:
operation RotateByPiOverTwo() : Unit {
    using (q = Qubit()) {
        let input = (PI() / 2.0, q);
        Ry(input);
        DumpMachine();
        Adjoint Ry(input);
    }
}

RotateByPiOverTwo

In [103]:
%simulate RotateByPiOverTwo

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865475 + 0𝑖

()

## Types in Q#: Arrays

For any type `'T`, the type `'T[]` represents _arrays_ where each element has type `'T`.

In [104]:
function FavoriteNumbers() : Int[] {
    return [1, 42, 101];
}

FavoriteNumbers

In [105]:
function ExampleData() : (Int, Bool[])[] {
    return [
        (3, [true, false, true]),
        (11, [false, false])
    ];
}

ExampleData

In [106]:
%simulate ExampleData

(3, [True,False,True]), (11, [False,False])

Really, _any_ type.

In [107]:
open Microsoft.Quantum.Arrays;

function Silly() : Unit[] {
    return ConstantArray(4, ());
}

Silly

In [108]:
%simulate Silly

(), (), (), ()

Like all other values in Q#, arrays are **immutable**; you can't change elements of an array after it has been created.


You can, however, use the copy-and-update operator `w/` to do something very similar:

In [109]:
function ElementaryVector(length : Int, index : Int) : Int[] {
    return ConstantArray(length, 0) // [0, 0, ..., 0]
           w/ index <- 1;           // replace the index'th element with 1.
}

ElementaryVector

In [110]:
%simulate ElementaryVector length=5 index=1

0, 1, 0, 0, 0

Just as with other values in Q#, making a copy with `w/` doesn't affect the original.

In [111]:
function ImmutabilityExample() : Unit {
    let data = [PauliI, PauliX, PauliY, PauliZ];
    let newData = data w/ 1 <- PauliZ;
    Message($"old: {data}\nnew: {newData}");
}

ImmutabilityExample

In [112]:
%simulate ImmutabilityExample

old: [PauliI,PauliX,PauliY,PauliZ]
new: [PauliI,PauliZ,PauliY,PauliZ]


()

You can combine `w/` and the `set` keyword to safely use arrays as mutable variables, without arrays themselves being mutable.

In [113]:
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arithmetic;

operation EstimateHistogram(nQubits : Int) : Int[] {
    mutable histogram = ConstantArray(PowI(2, nQubits), 0);
    for (idxSample in 0..99) {
        using (register = Qubit[nQubits]) {
            ApplyToEach(H, register);
            let index = MeasureInteger(LittleEndian(register));
            set histogram w/= index <- histogram[index] + 1;
        }
    }
    return histogram;
}

EstimateHistogram

In [116]:
%simulate EstimateHistogram nQubits=2

26, 25, 29, 20

## Types in Q#: Qubits

The `using` statement can be used to either ask for a single qubit, or an array of qubits.

Fresh qubits start off in the $|0\rangle$ state.

In [117]:
open Microsoft.Quantum.Diagnostics;

operation AllocateRegister(nQubits : Int) : Unit {
    using (register = Qubit[nQubits]) {
        DumpMachine();
    }
}

AllocateRegister

In [118]:
%simulate AllocateRegister nQubits=2

|00⟩	1 + 0𝑖

()

In [119]:
%simulate AllocateRegister nQubits=6

|000000⟩	1 + 0𝑖

()

## Control Flow in Q#:  `Adjoint` and `Controlled`

The `Adjoint` and `Controlled` keywords let you automatically undo or control Q# operations.

In [120]:
operation DemonstrateAdjoint(angle : Double) : Unit {
    using ((left, right) = (Qubit(), Qubit())) {
        Ry(angle, left);
        SWAP(left, right);
        Adjoint Ry(angle, right);
        DumpMachine();
    }
}

DemonstrateAdjoint

In [121]:
%simulate DemonstrateAdjoint angle=0.356

|00⟩	1 + 0𝑖

()

By adding `is Adj` to your operation definitions, you can use `Adjoint` with them as well as with built-in operations like `X` and `Ry`.

In [122]:
operation RotateAboutPlus(angle : Double, target : Qubit) : Unit is Adj {
    H(target);
    R1(angle, target);
    H(target);
}

RotateAboutPlus

In [123]:
operation DemonstrateRotationAboutPlus(angle : Double) : Unit {
    using (q = Qubit()) {
        RotateAboutPlus(angle, q);
        DumpMachine();
        Adjoint RotateAboutPlus(angle, q);
        DumpMachine();
    }
}

DemonstrateRotationAboutPlus

In [124]:
%simulate DemonstrateRotationAboutPlus angle=1.234

|0⟩	0.665232554035865 + 0.4719091046873169𝑖
|1⟩	0.3347674459641352 + -0.4719091046873169𝑖

|0⟩	1.0000000000000002 + 2.998097765362545E-17𝑖

()

`Controlled` lets you control quantum operations on another register being in the $|11\cdots 1\rangle$ state.

| Gate | Q# Operation | Q# Shorthand |
|---|---|---|
| $X$ | `X(target)` | - |
| CNOT | `Controlled X([control], target)` | `CNOT(control, target)` |
| CZ | `Controlled Z([control], target)` | `CZ(control, target)` |
| Toffoli | `Controlled X([control1, control2], target)` | `CCNOT(control1, control2, target)` |
| Fredkin | `Controlled SWAP([control], (target1, target2))` | - |


The `Controlled` keyword can be used with any operation that has `is Ctl` in its signature.

## Types in Q#: Functions and Operations as Values

With `Adjoint` and `Controlled` aside, we can come back to the last two kinds of types you'll commonly encounter in Q#:

functions and operations themselves.

In [125]:
operation ApplyTwiceCA<'T>(op : ('T => Unit is Adj + Ctl), input : 'T) : Unit is Adj + Ctl {
    op(input);
    op(input);
}

ApplyTwiceCA

The type `'T => Unit is Adj + Ctl` represents any operation that returns `Unit`, is adjointable, and controllable.

In [126]:
operation DoNothing(target : Qubit) : Unit is Adj + Ctl {
    ApplyTwiceCA(H, target);
}

DoNothing

### Partial application

Q# provides one more tool to make it easier to use functions and operations together.

If you give _part_ of the input to a function or operation, you get back a new function or operation that takes the remainder of the input.

In [127]:
open Microsoft.Quantum.Convert;

function PiOverNRotation(denominator : Int) : (Qubit => Unit is Adj + Ctl) {
    return Ry(PI() / IntAsDouble(denominator), _);
}

PiOverNRotation

In [128]:
operation DemonstratePiOverNRotation(denominator : Int) : Unit  {
    let rotateByPiOverN = PiOverNRotation(denominator);
    using (q = Qubit()) {
        rotateByPiOverN(q);
        DumpMachine();
        Adjoint rotateByPiOverN(q);
    }
}

DemonstratePiOverNRotation

In [129]:
%simulate DemonstratePiOverNRotation denominator=7

|0⟩	0.9749279121818236 + 0𝑖
|1⟩	0.2225209339563144 + 0𝑖

()

By passing operations to other functions and operations, and by using partial application, we can build powerful control patterns quickly.

In [130]:
operation AssertHTwiceDoesNothing() : Unit {
    AssertOperationsEqualReferenced(1,
        ApplyToFirstQubitCA(ApplyTwiceCA(H, _), _),
        ApplyToFirstQubitCA(I, _)
    );
}

AssertHTwiceDoesNothing

In [131]:
%simulate AssertHTwiceDoesNothing

()

## Control Flow in Q#: `within` and `apply`

Patterns like $U^\dagger V U$ are very common in quantum computing, and are specially supported in Q#.

In [132]:
operation ReflectAboutUniformSuperposition(register : Qubit[]) : Unit is Adj + Ctl {
    within {
        // Map |++⋯+⟩ to |00⋯0⟩.
        ApplyToEachCA(H, register);
        // Map |00⋯0⟩ to |11⋯1⟩.
        ApplyToEachCA(X, register);
    } apply {
        // Reflect about |11⋯1⟩.
        Controlled Z(Most(register), Tail(register));
    }
    // The mappings |00⋯0⟩ → |11⋯1⟩ and |++⋯+⟩ → |00⋯0⟩ are automatically
    // undone after the apply block.
}

ReflectAboutUniformSuperposition

In [133]:
operation DemoReflection() : Unit {
    using (register = Qubit[3]) {
        within {
            ReflectAboutUniformSuperposition(register);
        } apply {
            DumpMachine();
        }
    }
}

DemoReflection

In [134]:
%simulate DemoReflection

|000⟩	0.7500000000000002 + 0𝑖
|001⟩	-0.25000000000000006 + 0𝑖
|010⟩	-0.25000000000000006 + 0𝑖
|011⟩	-0.25000000000000006 + 0𝑖
|100⟩	-0.25000000000000006 + 0𝑖
|101⟩	-0.25000000000000006 + 0𝑖
|110⟩	-0.25000000000000006 + 0𝑖
|111⟩	-0.25000000000000006 + 0𝑖

()

## Control Flow in Q#: The Q# Standard Library

The Q# standard library provides a wide range of different functions and operations to help make writing out control flow patterns easier.

In [135]:
operation ApplyParity(controls : Qubit[], target : Qubit) : Unit is Adj + Ctl {
    ApplyToEachCA(CNOT(_, target), controls);
}

ApplyParity

In [136]:
operation DemonstrateParity(nQubits : Int) : Unit {
    using (register = Qubit[nQubits]) {
        ApplyToEach(H, register);
        using (target = Qubit()) {
            within {
                ApplyParity(register, target);
            } apply {
                Z(target);
            }
        }
        DumpMachine();
        ResetAll(register);
    }
}

DemonstrateParity

In [137]:
%simulate DemonstrateParity nQubits=3

|000⟩	0.35355339059327384 + 0𝑖
|001⟩	-0.35355339059327384 + 0𝑖
|010⟩	-0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	-0.35355339059327384 + 0𝑖
|101⟩	0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	-0.35355339059327384 + 0𝑖

()

In [138]:
operation ApplyPairwiseParity(controls : Qubit[], targets : Qubit[]) : Unit is Adj + Ctl {
    ApplyToEachCA(CCNOT, Zip3(Most(controls), Rest(controls), targets));
}

ApplyPairwiseParity

In [139]:
operation DemonstratePairwiseParity(nQubits : Int) : Unit {
    using ((controls, targets) = (Qubit[nQubits], Qubit[nQubits - 1])) {
        ApplyToEach(H, controls);
        ApplyPairwiseParity(controls, targets);
        DumpMachine();
        ResetAll(controls + targets);
    }
}

DemonstratePairwiseParity

In [140]:
%simulate DemonstratePairwiseParity nQubits=3

|00000⟩	0.35355339059327384 + 0𝑖
|00100⟩	0.35355339059327384 + 0𝑖
|01000⟩	0.35355339059327384 + 0𝑖
|01101⟩	0.35355339059327384 + 0𝑖
|10000⟩	0.35355339059327384 + 0𝑖
|10100⟩	0.35355339059327384 + 0𝑖
|11010⟩	0.35355339059327384 + 0𝑖
|11111⟩	0.35355339059327384 + 0𝑖

()

## Control Flow in Q#: Conditional Statements

While the `Controlled` keyword can be used to write out quantum-controlled operations, classical control flow can be represented using `if` statements:

In [141]:
operation TeleportMessage(msg : Qubit, there : Qubit) : Unit {
    using (here = Qubit()) {
        PrepareEntangledPair(here, there);
        Adjoint PrepareEntangledPair(msg, here);
        
        if (M(here) == One) { X(there); }
        if (M(msg) == One)  { Z(there); }
    }
}

TeleportMessage

You can also use the `?|` operator to pick one of two different values based on a condition.

In [142]:
operation TeleportMessage(msg : Qubit, there : Qubit) : Unit {
    using (here = Qubit()) {
        PrepareEntangledPair(here, there);
        Adjoint PrepareEntangledPair(msg, here);
        
        (M(here) == One ? X | I)(there);
        (M(msg) == One  ? Z | I)(there);
    }
}

TeleportMessage

## Control Flow in Q# ##

Other Control Flow Features:

- `while` loops in functions
- `repeat`/`until`/`fixup` loops in operations
- `fail` statements for error reporting

Check out https://docs.microsoft.com/quantum/user-guide/using-qsharp/control-flow.

## Diagnosing Q# Programs

Q# provides a variety of tools for diagnosing quantum programs:


- **Dump** instructions print out internal simulator state, and do nothing on hardware.
- **Facts** check classical conditions like equality between values.
- **Assertions** check conditions about quantum states when used on simulators, and do nothing on hardware.

In [143]:
%package Microsoft.Quantum.Standard.Visualization

Adding package Microsoft.Quantum.Standard.Visualization: done!

Microsoft.Quantum.Standard::0.12.20082513, QSharpCommunity.Simulators.Chp::0.1.7, Microsoft.Quantum.Standard.Visualization::0.12.20082513

In [144]:
operation DumpPrepareEntangledPair() : Unit {
    DumpOperation(2,
        ApplyToFirstTwoQubitsCA(PrepareEntangledPair, _)
    );
}

DumpPrepareEntangledPair

In [145]:
%simulate DumpPrepareEntangledPair

Real:
[[0.7071067811865477, 0.7071067811865477, 0, 0], 
[0, 0, 0.7071067811865477, -0.7071067811865477], 
[0, 0, 0.7071067811865477, 0.7071067811865477], 
[0.7071067811865477, -0.7071067811865477, 0, 0]]
Imag:
[[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, 0]]

()

## Putting it Together: Grover's Search

In [146]:
open Microsoft.Quantum.Samples.IEEEQuantumWeek;

In [147]:
operation MarkItemByIndex(
    idxMarkedItem : Int,
    register : Qubit[],                                             
    flag : Qubit)                                                   
: Unit is Adj + Ctl {
    (ControlledOnInt(idxMarkedItem, X))(register, flag);
}

MarkItemByIndex

In [148]:
operation DemoMarkedItem(idxMarkedItem : Int) : Unit {
    using ((register, flag) = (Qubit[3], Qubit())) {
        within {
            ApplyToEachCA(H, register);
            MarkItemByIndex(idxMarkedItem, register, flag);
        } apply {
            DumpMachine();
        }
    }
}

DemoMarkedItem

In [149]:
%simulate DemoMarkedItem idxMarkedItem=3

|0000⟩	0.35355339059327384 + 0𝑖
|0010⟩	0.35355339059327384 + 0𝑖
|0100⟩	0.35355339059327384 + 0𝑖
|0110⟩	0.35355339059327384 + 0𝑖
|1000⟩	0.35355339059327384 + 0𝑖
|1010⟩	0.35355339059327384 + 0𝑖
|1101⟩	0.35355339059327384 + 0𝑖
|1110⟩	0.35355339059327384 + 0𝑖

()

In [150]:
operation SearchForMarkedItem(
    nItems : Int,
    markItem : ((Qubit[], Qubit) => Unit is Adj)
)
: Int {
    using (qubits = Qubit[BitSizeI(nItems - 1)]) {
        ApplyToEachCA(H, qubits);

        DumpMachine();

        for (idxIteration in 0..NIterations(BitSizeI(nItems - 1)) - 1) {
            ReflectAboutMarkedState(markItem, qubits);
            ReflectAboutInitialState(ApplyToEachCA(H, _), qubits);        
        }

        DumpMachine();

        return MeasureInteger(LittleEndian(qubits));
    }
}

SearchForMarkedItem

In [151]:
operation RunGroverSearch(nItems : Int, idxMarkedItem : Int) : Unit {
    let markItem = MarkItemByIndex(idxMarkedItem, _, _);
    let foundItem = SearchForMarkedItem(nItems, markItem);
    Message($"Marked {idxMarkedItem} and found {foundItem}.");
}

RunGroverSearch

In [152]:
%simulate RunGroverSearch nItems=4 idxMarkedItem=1

|00⟩	0.5000000000000001 + 0𝑖
|01⟩	0.5000000000000001 + 0𝑖
|10⟩	0.5000000000000001 + 0𝑖
|11⟩	0.5000000000000001 + 0𝑖

|10⟩	-1.0000000000000007 + 0𝑖

Marked 1 and found 1.


()